In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

In [54]:
loader = CSVLoader('data_input/test.csv')
big_df = loader.load()

In [55]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True
)

In [56]:
chunks = text_splitter.split_documents(big_df)

In [57]:
embedings = OllamaEmbeddings(
    model='all-minilm'
)

In [58]:
chromadb = Chroma.from_documents(chunks, embedings, persist_directory = 'vector_db', collection_name= 'vectors_test')

In [59]:
# Prompt
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Conversation_history: {conversation_history}

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [60]:
# Chain
MODEL = 'llama2'
model = Ollama(model=MODEL)
parser = StrOutputParser()
retriever = chromadb.as_retriever()

from operator import itemgetter

# chain = (
#     {
#         "context": itemgetter("question") | retriever,
#         "conversation_history": itemgetter("conversation_history"),
#         "question": itemgetter("question"),
#     }
#     | prompt
#     | model
#     | parser
# )

In [69]:
query = 'tell us about the Bulldog messenger near Liege?'

In [70]:
retriever = chromadb.as_retriever()

In [71]:
answer = retriever.invoke(query)

In [72]:
answer

[Document(page_content='url: https://lenta.ru/news/1914/09/16/lermontov/\ntitle: 1914. Празднование столетия М.Ю. Лермонтова отложено\ntext: Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени.  \n«Русский инвалид», 16 сентября 1914 года.\ntopic: Библиотека\ntags: Первая мировая\ndate: 1914/09/16', metadata={'row': 1, 'source': 'data_input/test.csv', 'start_index': 0}),
 Document(page_content='url: https://lenta.ru/news/1999/09/06/kama/\ntitle: В Каму вытекло 100\xa0тонн мазута\ntext: В результате произошедшего у пристани Дербешки (Татарстан) в ночь с субботы на воскресенье столкновения грузового теплохода "ОТА-855", груженого технической солью, и баржи "Бельская-65" с нефтепродуктами, в Каму вытекло более ста тонн мазута, - сообщает ИТАР-Т

In [62]:

history = ''

answer = chain.invoke({'question': query, "conversation_history": history})
print(answer)

As far as I know, there is no information available about a "Bulldog-hunter" in Lille or any other location. It is possible that you may have come across a misunderstanding or misinformation. Could you please provide more context or clarify what you are looking for?
